# 필요한 라이브러리 import

In [2]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [3]:
import joblib

midfielder_model = joblib.load('./models/midfielder_lgbm.pkl')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [5]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']

In [8]:
num_col = [midfielder_df.columns[i] for i,x in enumerate(midfielder_df.dtypes) if x == 'int64' or x == 'float64']
independent_value = set(num_col) - set(['평균획득승점/경기','출장시간','승','무','패','경기당 팀 득점/90','경기당 팀 실점/90'])
dependent_value = '평균획득승점/경기'

In [10]:
midfielder_df['예측PPG']=midfielder_model.predict(midfielder_df.loc[:,independent_value])

In [11]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
12,Miguel Almirón,2021-2022,미드필더,뉴캐슬,잉글랜드 프리미어 리그,12,4,0,19,6,...,66,31.98,27.82,1.24,1.27,1.02,4.08,1.57,1.66,3.205205
28,Harvey Barnes,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,14,7,0,22,14,...,86,34.44,30.42,1.65,2.16,3.19,4.05,5.62,1.45,3.199270
167,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,13,9,0,26,4,...,83,35.92,31.41,1.62,2.81,3.33,4.70,5.58,1.91,3.186302
254,Mason Mount,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,17,6,0,36,11,...,82,40.77,35.28,1.91,1.71,1.57,4.32,6.23,2.02,3.182523
166,Martin Ødegaard,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,15,11,0,26,7,...,75,47.66,41.45,1.65,1.07,1.37,2.97,4.16,1.89,3.181625


In [12]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

2.190137788352085

In [13]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [14]:
midfielder_war = midfielder_df.sort_values(by='WAR',ascending=False).copy()
midfielder_war = midfielder_war.reset_index(drop=True)
midfielder_war.head(50)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
0,Miguel Almirón,2021-2022,미드필더,뉴캐슬,잉글랜드 프리미어 리그,12,4,0,19,6,...,31.98,27.82,1.24,1.27,1.02,4.08,1.57,1.66,3.205205,2.015067
1,Harvey Barnes,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,14,7,0,22,14,...,34.44,30.42,1.65,2.16,3.19,4.05,5.62,1.45,3.199270,2.009133
2,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,13,9,0,26,4,...,35.92,31.41,1.62,2.81,3.33,4.70,5.58,1.91,3.186302,1.996164
3,Mason Mount,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,17,6,0,36,11,...,40.77,35.28,1.91,1.71,1.57,4.32,6.23,2.02,3.182523,1.992386
4,Martin Ødegaard,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,15,11,0,26,7,...,47.66,41.45,1.65,1.07,1.37,2.97,4.16,1.89,3.181625,1.991487
5,Kevin De Bruyne,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,11,22,0,33,11,...,53.03,47.70,1.54,1.08,0.95,2.33,0.91,1.96,3.172958,1.982820
6,Nikola Vlašić,2021-2022,미드필더,웨스트 햄,잉글랜드 프리미어 리그,14,5,0,20,16,...,37.78,32.50,2.00,0.88,1.48,3.71,1.37,1.43,3.167574,1.977436
7,Bernardo Silva,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,15,9,0,26,11,...,40.24,35.38,1.82,1.42,2.39,3.34,4.92,1.85,3.100216,1.910078
8,Saúl,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,9,14,21/7/5 - 22/6/30,31,6,...,51.83,45.79,1.43,2.12,1.00,5.15,0.67,2.11,3.080231,1.890094
9,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,5,6,0,21,5,...,39.14,33.95,1.62,3.37,1.75,5.60,3.89,2.19,3.079165,1.889027


In [16]:
midfielder_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(40)

,이름,시즌,골,도움,WAR
0,Miguel Almirón,2021-2022,12,4,2.015067
1,Harvey Barnes,2021-2022,14,7,2.009133
2,Bukayo Saka,2021-2022,13,9,1.996164
3,Mason Mount,2021-2022,17,6,1.992386
4,Martin Ødegaard,2021-2022,15,11,1.991487
5,Kevin De Bruyne,2021-2022,11,22,1.982820
6,Nikola Vlašić,2021-2022,14,5,1.977436
7,Bernardo Silva,2021-2022,15,9,1.910078
8,Saúl,2021-2022,9,14,1.890094
9,Hakim Ziyech,2021-2022,5,6,1.889027
